In [12]:
import os
import pandas as pd

In [23]:
f = open("tickers.csv")
tickers = f.read().split(' Equity\r\n')
f.close()
tickers = tickers[:-1]
tickers[0] = tickers[0][2:]

In [24]:
tickers = [t[:-3].lower()for t in tickers if t[-2] == 'U' and t[-1] == 'S']
existing_tickers = [t for t in tickers if os.path.exists('Stocks/'  + t + '.us.txt')]

In [25]:
print 'original tickers: ' + str(len(tickers))
print 'tickers that we have data for: ' + str(len(existing_tickers))
print 'last date: 2017-11-10'

original tickers: 156
tickers that we have data for: 154
last date: 2017-11-10


In [26]:
insider_trades = pd.read_csv('insider_trades.csv')#, index_col=0)
insider_trades['min_date'] = pd.to_datetime(insider_trades['min_date'])
insider_trades['max_date'] = pd.to_datetime(insider_trades['max_date'])
last_date = pd.to_datetime('2017-11-10')
cutoff = pd.to_datetime('2017-10-10')

In [27]:
#for t in existing_tickers:
#    data = open('Stocks/' + t + '.us.txt')
#    last_row = ''
#    for line in data:
#        last_row = line(
#    print last_row)

In [28]:
# Only keep ticker if its max & min date are before the cutoff, and we have its stock price data
trades_filtered = insider_trades[insider_trades['min_date'] <= cutoff]
trades_filtered = trades_filtered[trades_filtered['max_date'] <= cutoff]
trades_filtered['ticker'] = trades_filtered['ticker'].str[:-10]
trades_filtered['ticker'] = [str(i).lower() for i in trades_filtered['ticker']]
trades_filtered = trades_filtered[trades_filtered['ticker'].isin(existing_tickers)]

In [29]:
print insider_trades.shape
print trades_filtered.shape
#trades_filtered

(170, 5)
(83, 5)


In [30]:
for ticker in trades_filtered['ticker']:
    data = pd.read_csv('Stocks/' + ticker + '.us.txt')
    data.drop(['Open', 'High', 'Low', 'Volume', 'OpenInt'], axis = 1,inplace=True)
    data.set_index('Date', inplace=True)
    data.to_csv('data/' + ticker + '.csv')

In [31]:
trades_filtered.set_index('ticker', inplace=True)
trades_filtered.to_csv("filtered_trades.csv")